**Rigorous Approach for Training Unconstrained Gaussian Radial Basis Function (RBF) Networks with Deterministic Center Distribution**

This code demonstrates the *Approximate* training algorithm developed for standalone unconstrained RBF network models for nonlinear transient chemical process systems. The structure of the RBF model is optimized by varying the number of nodes / neurons / centers in hidden layer based on a recursive approach for minimizing the corrected Akaike Information Criteria (AICc). 

Two different types of center distributions are chosen for Gaussian RBF networks, i.e., **deterministic (fixed)** and probabilistic (stochastic). While the deterministic distribution refers to assigning constant values to the centers / widths of the RBFs, the stochastic distribution considers sampling the centers from Gaussian distributions having the same mean and variances as the model inputs. The solution space is parameterized by representing the RBF centers / widths and the means / variances of the center distributions as additional model parameters to be estimated during training of the optimal RBF network models with deterministic and stochastic centers, respectively, in addition to the connection weights in the output layer of the network models. 

In the *Rigorous* approach, the number of RBF nodes (centers) in hidden layer as well as the corresponding optimal coordinates of the RBF centers / widths are evaluated based on solving an optimization (MINLP) problem using Bidirectional Branch and Bound (BB&B) algorithm coupled with IPOPT (for deterministic centers) or Adam SGD with Minibatch (for probabilistic centers) with the objective of minimizing the corrected Akaike Information Criteria (AICc).

*Load the training and validation datasets and specify the input and output variables for the RBF models. Note that the user can consider any dynamic dataset for training and validation. The rows signify the time steps for transient data and the columns signify the input and output variables.*

*The nonlinear dynamic continuous stirred tank reactor (CSTR) system is chosen for demonstration of the proposed approach.*

In [1]:
import numpy as np
import scipy as sp
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from idaes.core.solvers import get_solver
get_solver()
import matplotlib.pyplot as plt
import pandas as pd
from pyDOE import *
import math as mt
import time
import json
import pickle

In [3]:
# Loading the data for model development

data = pd.read_excel("Dynamic CSTR Data.xlsx","Data", header=None).values
data = data[2:1202, 1:]

# For this specific system, the first five columns are the model inputs and the following four columns are the model outputs

input_data = data[:,0:5]
output_data = data[:,5:]

In [4]:
# Defining optimization problem in Pyomo for estimation of coordinates of RBF centers / widths

def GRBF_optim(tc,tn,ni,no,Imat_t,dsr_t):
    
    M = pyo.ConcreteModel()

    M.I1 = pyo.RangeSet(1, ni)
    M.I2 = pyo.RangeSet(1, tc)
    M.I3 = pyo.RangeSet(1, 1)
    M.I4 = pyo.RangeSet(1, no)
    M.I5 = pyo.RangeSet(1, tn)
    
    M.x1 = pyo.Var(M.I1,M.I2,bounds = (1e-3,2.5), initialize = 0.5)   # centermat
    M.x2 = pyo.Var(M.I3, bounds = (1e-3,4), initialize = 1)           # sigma
    M.x3 = pyo.Var(M.I2,M.I4,bounds = (-1e5,1e5))                    # z
    
    M.y2 = pyo.Var(M.I2,M.I5)                     # PhiofD
    M.y3 = pyo.Var(M.I5,M.I4)                     # yRBF
    
    @M.Expression(M.I2, M.I5)
    def D(M,i,j):
        return pyo.sqrt(sum((Imat_t[k-1,j-1] - M.x1[k,i])**2 for k in M.I1))   
    
    def constraint_rule_2(M,i,j):
        return M.y2[i,j] == (1/pyo.sqrt(2*mt.pi*M.x2[1]**2))*pyo.exp(-(M.D[i,j] * M.D[i,j])/(2*M.x2[1]**2)) 
    
    M.constraint_2 = pyo.Constraint(M.I2, M.I5, rule = constraint_rule_2)
    
    def constraint_rule_3(M,i,j):
        return M.y3[i,j] == sum((M.y2[k,i] * M.x3[k,j]) for k in M.I2)
    
    M.constraint_3 = pyo.Constraint(M.I5, M.I4, rule = constraint_rule_3)
    
    def GRBF_optim_det(M):        
        
        dsr_tt = dsr_t.T        
        obj_value = sum(sum((dsr_tt[i-1,j-1] - M.y3[i,j]) ** 2 for i in M.I5) for j in M.I4)
        return obj_value
    
    M.obj = pyo.Objective(rule = GRBF_optim_det, sense = pyo.minimize)
    
    solver = pyo.SolverFactory('ipopt')
    solver.options['max_iter'] = 200
    
    results = solver.solve(M, tee = True)
    
    # Accessing the optimal solutions
    
    centermat = np.zeros((ni,tc))
    for (i,j) in M.x1:
        centermat[i-1,j-1] = pyo.value(M.x1[i,j])
    
    sigma = pyo.value(M.x2[1])
    
    z = np.zeros((tc,no))
    for (i,j) in M.x3:
        z[i-1,j-1] = pyo.value(M.x3[i,j])

    D = np.zeros((tc,tn))
    for i, c in enumerate(centermat.T):
        D[i,:] =  np.linalg.norm(Imat_t.T - c, 2, axis = 1)
    
    PhiofD = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(D*D)/(2*sigma**2))
    yRBF = np.dot(PhiofD.T,z)
    dsr_tt = dsr_t.T

    eps = np.transpose(dsr_tt - yRBF)
    R = 0.1*np.eye(no)
    
    k = ni*tc + 1 + z.shape[0]*z.shape[1]
    
    n_AIC = tn; m_AIC = no;
    
    llh = (-n_AIC*m_AIC/2)*np.log(2*np.pi) - 0.5*n_AIC*np.log(np.linalg.det(R)) - 0.5*np.dot(np.dot(np.sum(eps, axis=1), \
                np.linalg.inv(R)),  np.transpose(np.sum(eps, axis=1)))
    
    obj_AICc = -2*llh + 2*k + 2*k*(k+1)/(n_AIC - k - 1)
    Term1 = -2*llh
    Term2 = 2*k + 2*k*(k+1)/(n_AIC - k - 1)

    return [obj_AICc,Term1,Term2]

**Bidirectional Branch and Bound Algorithm**

The objective function under consideration is given by:
$$
    AIC_c = -2*\log(L(\theta)) + 2*k + \frac{2*k*(k+1)}{n-k-1}
$$
where, $k$ denotes the number of model parameters. 

The first-term on the RHS is referred to as $Term_1$, whereas the last two terms are collectively referred to as $Term_2$. While the analytical derivatives of $Term_2$ can be readily evaluated with respect to the number of RBF centers ($n_c$), the same cannot be evaluated for $Term_1$.

Therefore, $Term_1$ is approximated as a quadratic interpolation in the neighborhood of $n_c$ at every iteration. The symbolic expressions of the objective function as well as the gradient and hessian of the objective function are evaluated and converted to Python functions for use in subsequent code chunks.

In [ ]:
from sympy import symbols, diff, hessian, lambdify

# Define symbolic variables
nc_s, a_s, b_s, c_s, ni_s, no_s, n_s = symbols('nc_s a_s b_s c_s ni_s no_s n_s')

# Define the Objective Function
Term1 = a_s*nc_s**2 + b_s*nc_s + c_s
k = (no_s + ni_s)*nc_s + 1
Term2 = 2*k + 2*k*(k+1)/(n_s - k - 1)
obj = Term1 + Term2

# Compute the gradient and hessian of obj
grad_obj = diff(obj, nc_s)
hess_obj = diff(grad_obj, nc_s)

# Convert symbolic expressions to Python functions
obj_f = lambdify((nc_s, a_s, b_s, c_s, ni_s, no_s, n_s), obj, 'numpy')
grad_obj_f = lambdify((nc_s, a_s, b_s, c_s, ni_s, no_s, n_s), grad_obj, 'numpy')
hess_obj_f = lambdify((nc_s, a_s, b_s, c_s, ni_s, no_s, n_s), hess_obj, 'numpy')

In [ ]:
# Defining Newton's method of Backtracking Line Search
def Newton_opti(a,b,c,n,ni,no,obj_f,grad_obj_f,hess_obj_f,tc_start,step_tc):
    
    # Initialize number of iterations
    k = 0

    x = []; fx = []; gradfx = []; hessfx = []; p = []; alpha = [];

    # Specification of the initial guess of the decision variable(s)
    x = np.append(x,tc_start)
    fx = np.append(fx,obj_f(x[k],a,b,c,ni,no,n))
    gradfx = np.append(gradfx,grad_obj_f(x[k],a,b,c,ni,no,n))
    hessfx = np.append(hessfx,hess_obj_f(x[k],a,b,c,ni,no,n))

    # Checking whether Hessian is positive-definite, otherwise regularization

    while not (hessfx[k] >= 0):
        hessfx[k] += 0.1*np.eye(1)

    # Evaluating p(k) using linear solver
    p = np.append(p,-gradfx[k]/hessfx[k])
    
    # Initialization of the step length
    alpha = np.append(alpha,0.1)
    
    # Constants for updating alpha and parameter involved in Armijo condition
    tau = 0.4;        # for updating alpha = tau*alpha
    gamma = 1e-4;     # scalar parameter value in Armijo condition

    # Updating the value of 'x' using x(k+1) = x(k) + alpha(k)*p(k)
    x = np.append(x,x[k] + alpha[k]*p[k])
    
    # Generating f(x) and gradf(x) values for the (k+1)-th iteration
    fx = np.append(fx,obj_f(x[k+1],a,b,c,ni,no,n))
    gradfx = np.append(gradfx,grad_obj_f(x[k+1],a,b,c,ni,no,n))

    # Checking the Goldstein-Armijo condition
    while not (fx[k]+(1-gamma)*alpha[k]*gradfx[k]*p[k] <= fx[k+1] and fx[k+1] <= fx[k]+gamma*alpha[k]*gradfx[k]*p[k]):
        while not (x[k+1] > (tc_start - step_tc) and x[k+1] < (tc_start + step_tc)):
            alpha[k] = alpha[k]*tau
            x[k+1] = x[k] + alpha[k]*p[k]
            fx[k+1] = obj_f(x[k+1],a,b,c,ni,no,n)
            gradfx[k+1] = grad_obj_f(x[k+1],a,b,c,ni,no,n)
            

    hessfx = np.append(hessfx,hess_obj_f(x[k+1],a,b,c,ni,no,n))

    # Calculating the different tolerances as termination criteria for the optimization loop

    relxtol = np.linalg.norm(x[k+1]-x[k])/np.linalg.norm(x[k])   # Rel tol for variable(s) 'x'
    eps = 1e-3

    absfxtol = np.linalg.norm(fx[k+1] - fx[k])      # Abs tol for f(x)
    eps1 = 1e-3

    absgfxtol = np.linalg.norm(gradfx[k])           # Abs tol for grad(f(x))
    eps2 = 1e-3

    while (relxtol>eps or absfxtol>eps1 or absgfxtol>eps2) and (x[k+1] > (tc_start - step_tc) and x[k+1] < (tc_start + step_tc)):

        k += 1
        p = np.append(p,-gradfx[k]/hessfx[k])
        alpha = np.append(alpha,alpha[k-1])
        x = np.append(x,x[k] + alpha[k]*p[k])
        fx = np.append(fx,obj_f(x[k+1],a,b,c,ni,no,n))
        gradfx = np.append(gradfx,grad_obj_f(x[k+1],a,b,c,ni,no,n))

        # Checking the Armijo-Goldstein Condition
        while not (fx[k]+(1-gamma)*alpha[k]*gradfx[k]*p[k] <= fx[k+1] and fx[k+1] <= fx[k]+gamma*alpha[k]*gradfx[k]*p[k]):
            while not (x[k+1] > (tc_start - step_tc) and x[k+1] < (tc_start + step_tc)):
                alpha[k] = alpha[k]*tau
                x[k+1] = x[k] + alpha[k]*p[k]
                fx[k+1] = obj_f(x[k+1],a,b,c,ni,no,n)
                gradfx[k+1] = grad_obj_f(x[k+1],a,b,c,ni,no,n)

        hessfx = np.append(hessfx,hess_obj_f(x[k+1],a,b,c,ni,no,n))
        # Checking whether Hessian is positive-definite, otherwise regularization
        while not (hessfx[k] >= 0):
            hessfx[k] += 0.1*np.eye(hessfx[k].shape[0])

        # Recalculating Termination Criteria with updated values
        relxtol = np.linalg.norm(x[k+1]-x[k])/np.linalg.norm(x[k])
        absfxtol = np.linalg.norm(fx[k+1] - fx[k])
        absgfxtol = np.linalg.norm(grad_obj_f(x[k],a,b,c,ni,no,n))
        

    nc_opt = x[-2]
    return nc_opt           

In [ ]:
# Development of BB&B algorithm and optimal estimation of centers, widths, and weights

def GRBF_det_cen_optim_BBB_AICc(input_data, output_data):
    data = np.concatenate((input_data, output_data), axis = 1)
    ni = input_data.shape[1]
    no = output_data.shape[1]
    nt = ni+no
    
    tt = data.shape[0]
    tn = int(np.floor(0.6*tt))
    
    # Normalizing the input and output variables

    norm_mat = np.zeros((tt,nt))
    delta = np.zeros((1,nt))
    for i in range(nt):
        delta[:,i] = max(data[:,i]) - min(data[:,i])
        norm_mat[:,i] = (data[:,i] - min(data[:,i]))/delta[0,i]

    Imat = norm_mat[:,0:ni].T
    dsr = norm_mat[:,ni:ni+no].T
    
    tr_steps = np.random.choice(tt, tn, replace=False)
    tr_steps = np.sort(tr_steps) 
    
    dsr_t = np.zeros((no,tn))
    Imat_t = np.zeros((ni,tn))
    
    for i in range(tn):
        ts = tr_steps[i]
        dsr_t[:,i] = dsr[:,ts]
        Imat_t[:,i] = Imat[:,ts]


    # Training Input: Imat_t
    # Training Output: dsr_t

    check = np.inf
    tc_opt_1 = []
    AICc_opt = []

    step_tc = 5; tc_start = 30;

    [AICc_1, Term1_1, Term2_1] = GRBF_optim(tc_start,tn,ni,no,Imat_t,dsr_t)

    while (tc_start > 0):
        x1 = tc_start - step_tc; x2 = tc_start + step_tc; x3 = tc_start;

        [AICc_q_1, Term1_q_1, Term2_q_1] = GRBF_optim(x1,tn,ni,no,Imat_t,dsr_t)
        [AICc_q_2, Term1_q_2, Term2_q_2] = GRBF_optim(x2,tn,ni,no,Imat_t,dsr_t)

        a0 = Term1_q_1; a1 = np.divide((Term1_q_2 - Term1_q_1),(x2 - x1));
        a2 = (1/(x3 - x2))*(((Term1_1 - Term1_q_1)/(x3 - x1)) - a1);

        # f(x) = a0 + a1*(x - x1) + a2*(x - x1)*(x - x2)
        a = a2; b = a1 - a2*x1 - a2*x2; c = a0 - a1*x1 + a2*x1*x2;

        n = tn
      
        tc_opt = Newton_opti(a,b,c,n,ni,no,obj_f,grad_obj_f,hess_obj_f,tc_start,step_tc)
        
        obj_1 = obj_f(np.floor(tc_opt),a,b,c,ni,no,n)
        obj_2 = obj_f(np.ceil(tc_opt),a,b,c,ni,no,n)

        if (obj_1 <= obj_2):
            tc_opt_1.append(int(np.floor(tc_opt)))
        else:
            tc_opt_1.append(int(np.ceil(tc_opt)))

        [AICc_opt_1, Term1_opt_1, Term2_opt_1] = GRBF_optim(tc_opt_1[-1],tn,ni,no,Imat_t,dsr_t)        
        Term1_1 = Term1_opt_1
        
        AICc_opt = np.append(AICc_opt,AICc_opt_1)

        AICc_opt = np.array(AICc_opt)

        if (AICc_opt[-1] < AICc_1):
            AICc_1 = AICc_opt[-1]
            tc_start = tc_opt_1[-1]
        else:
            if (AICc_opt.shape[0] > 1):
                AICc_1 = AICc_opt[-2]
                tc_1 = tc_opt_1[-2]
            else:
                tc_1 = tc_opt_1[-1]

            tc_2 = tc_opt_1[-1]
            break

    if (tc_1 < tc_2):
        tc_v = np.arange(tc_1,tc_2+step_tc); tc_v = tc_v.T;
    else:
        tc_v = np.arange(tc_2,tc_1+step_tc); tc_v = tc_v.T;

    AICc_v = np.zeros((tc_v.shape[0],1))
    for j in range(tc_v.shape[0]):
        [AICc, Term1, Term2] = GRBF_optim(int(tc_v[j]),tn,ni,no,Imat_t,dsr_t)
        AICc_v[j] = AICc

    min_AICc = np.min(AICc_v)
    idx_min = np.argmin(AICc_v)

    num_of_centers = tc_v[idx_min]
    tc = num_of_centers

    # Setting up the optimization problem

    Mod = pyo.ConcreteModel()

    Mod.I1 = pyo.RangeSet(1, ni)
    Mod.I2 = pyo.RangeSet(1, tc)
    Mod.I3 = pyo.RangeSet(1, 1)
    Mod.I4 = pyo.RangeSet(1, no)
    Mod.I5 = pyo.RangeSet(1, tn)
    
    Mod.x1 = pyo.Var(Mod.I1,Mod.I2,bounds = (1e-3,2.5), initialize = 0.5)   # centermat
    Mod.x2 = pyo.Var(Mod.I3, bounds = (1e-3,4), initialize = 1)           # sigma
    Mod.x3 = pyo.Var(Mod.I2,Mod.I4,bounds = (-1e5,1e5))                    # z
    
    # Mod.y1 = pyo.Var(Mod.I2,Mod.I5)
    Mod.y2 = pyo.Var(Mod.I2,Mod.I5)                     # PhiofD
    Mod.y3 = pyo.Var(Mod.I5,Mod.I4)                     # yRBF
    
    @Mod.Expression(Mod.I2, Mod.I5)
    def D(Mod,i,j):
        return pyo.sqrt(sum((Imat_t[k-1,j-1] - Mod.x1[k,i])**2 for k in Mod.I1))   
    
    def constraint_rule_2(Mod,i,j):
        return Mod.y2[i,j] == (1/pyo.sqrt(2*mt.pi*Mod.x2[1]**2))*pyo.exp(-(Mod.D[i,j] * Mod.D[i,j])/(2*Mod.x2[1]**2)) 
    
    Mod.constraint_2 = pyo.Constraint(Mod.I2, Mod.I5, rule = constraint_rule_2)
    
    def constraint_rule_3(Mod,i,j):
        return Mod.y3[i,j] == sum((Mod.y2[k,i] * Mod.x3[k,j]) for k in Mod.I2)
    
    Mod.constraint_3 = pyo.Constraint(Mod.I5, Mod.I4, rule = constraint_rule_3)
    
    def GRBF_optim_det(Mod):        
        
        dsr_tt = dsr_t.T        
        obj_value = sum(sum((dsr_tt[i-1,j-1] - Mod.y3[i,j]) ** 2 for i in Mod.I5) for j in Mod.I4)
        return obj_value
    
    Mod.obj = pyo.Objective(rule = GRBF_optim_det, sense = pyo.minimize)
    
    solver = pyo.SolverFactory('ipopt')
    solver.options['max_iter'] = 200
    
    results = solver.solve(Mod, tee = True)
    
    # Accessing the optimal solutions
    
    centermat = np.zeros((ni,tc))
    for (i,j) in Mod.x1:
        centermat[i-1,j-1] = pyo.value(Mod.x1[i,j])
    
    sigma = pyo.value(Mod.x2[1])
    
    z = np.zeros((tc,no))
    for (i,j) in Mod.x3:
        z[i-1,j-1] = pyo.value(Mod.x3[i,j])

    D = np.zeros((tc,tn))
    for i, c in enumerate(centermat.T):
        D[i,:] =  np.linalg.norm(Imat_t.T - c, 2, axis = 1)
    
    PhiofD = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(D*D)/(2*sigma**2))
    yRBF = np.dot(PhiofD.T,z)
    dsr_tt = dsr_t.T

    eps = np.transpose(dsr_tt - yRBF)
    R = 0.1*np.eye(no)
    
    k = ni*tc + 1 + z.shape[0]*z.shape[1]
    
    n_AIC = tn; m_AIC = no;
    

    llh = (-n_AIC*m_AIC/2)*np.log(2*np.pi) - 0.5*n_AIC*np.log(np.linalg.det(R)) - 0.5*np.dot(np.dot(np.sum(eps, axis=1), \
                np.linalg.inv(R)),  np.transpose(np.sum(eps, axis=1)))

    AICc = -2*llh + 2*k + 2*k*(k+1)/(n_AIC - k - 1)
           
    if (AICc < check):
        check = AICc
        z_f = z
        centermat_f = centermat
        sigma_f = sigma
        num_of_centers = tc

    z = z_f; centermat = centermat_f; sigma = sigma_f;
    
    return [z,centermat,sigma,num_of_centers]

In [ ]:
# Running the code to estimate the optimal parameters
[z,centermat,sigma,num_of_centers] = GRBF_det_cen_optim_BBB_AICc(input_data, output_data)

In [ ]:
# Generate the results for entire dataset

data = np.concatenate((input_data, output_data), axis = 1)
ni = input_data.shape[1]
no = output_data.shape[1]
nt = ni+no
    
tt = data.shape[0]
    
# Normalizing the input and output variables

norm_mat = np.zeros((tt,nt))
delta = np.zeros((1,nt))
for i in range(nt):
    delta[:,i] = max(data[:,i]) - min(data[:,i])
    norm_mat[:,i] = (data[:,i] - min(data[:,i]))/delta[0,i]

in_norm = norm_mat[:,0:ni]; in_norm = in_norm.transpose();
out_norm = norm_mat[:,ni:ni+no]; out_norm = out_norm.transpose();

dsr_v = out_norm; Imat_v = in_norm;

D = np.zeros((num_of_centers,tt))
for i, c in enumerate(centermat.T):
    D[i,:] =  np.linalg.norm(Imat_v.T - c, 2, axis = 1)

yHL = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(D*D)/(2*sigma**2))

yRBF = np.dot(yHL.T,z)

target_data = np.zeros((tt,no))
yRBF_unnorm = np.zeros((tt,no))

for i in range(no):
    target_data[:,i] = np.transpose(dsr_v[i])*delta[0,ni+i] + min(data[:,ni+i])
    yRBF_unnorm[:,i] = yRBF[:,i]*delta[0,ni+i] + min(data[:,ni+i])

In [ ]:
# Plotting Results

plt.rcParams.update(
    {
        "figure.max_open_warning": 0,
        "axes.titlesize": 16,
        "axes.labelsize": 12,
        "axes.linewidth": 2,
        "lines.linewidth": 3,
        "lines.markersize": 10,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "savefig.bbox": "tight",
        "legend.fontsize": "medium",
    }
)

plt.subplot(2,2,1)
plt.plot(target_data[:,0], color = 'blue', linewidth = 1.5, label = 'Measurements')
plt.plot(yRBF_unnorm[:,0], color = 'red', linewidth = 1.2, linestyle = '--', label = 'RBF Model')
plt.xlim(0, target_data.shape[0])
plt.xlabel('Time (mins)', fontname='Times New Roman')
plt.ylabel('$C_A$ ($kmol/m^3$)')
plt.ylim(35,100)
plt.title("(a)", fontname='Times New Roman')
plt.legend(loc='upper right',prop={'family': 'Times New Roman', 'size': 12})
plt.subplot(2,2,2)
plt.plot(target_data[:,1], color = 'blue', linewidth = 1.5, label = 'Measurements')
plt.plot(yRBF_unnorm[:,1], color = 'red', linewidth = 1.2, linestyle = '--', label = 'RBF Model')
plt.xlim(0, target_data.shape[0])
plt.xlabel('Time (mins)', fontname='Times New Roman')
plt.ylabel('$C_B$ ($kmol/m^3$)')
plt.ylim(35,100)
plt.title("(b)", fontname='Times New Roman')
plt.legend(loc='upper right',prop={'family': 'Times New Roman', 'size': 12})

plt.tight_layout()
plt.show()